In [272]:
from utils.base import *
from utils.sarimax import sarimax_training, predict_target

In [324]:
basepath = 'datasci_dataset_2022'
province = ['BKK', 'Chiangmai', 'Khonkaen', 'Rayong', 'Saraburi', 'Surat'] # Chiangmai, Khonkaen, Saraburi
idx_province = 4
folderpath = os.path.join(basepath, province[idx_province])

mode = 'train'
train_data = pd.read_csv(os.path.join(folderpath, f"{mode}\\{mode}ing.csv")).rename({'Unnamed: 0': 'date_time'}, axis=1)
train_data['date_time'] = pd.to_datetime(train_data['date_time'])
train_data = train_data.set_index('date_time')
train_data = train_data.reindex(pd.date_range("2017-07-01 00:00:00", "2020-06-30 23:00:00",freq='6h'), fill_value=np.nan) # <= for Khonkaen and saraburi start at 2018

mode = 'test'
test_data = pd.read_csv(os.path.join(folderpath, f"{mode}\\{mode}ing.csv")).rename({'Unnamed: 0': 'date_time'}, axis=1)
test_data['date_time'] = pd.to_datetime(test_data['date_time'])
test_data = test_data.set_index('date_time')
test_data = test_data.reindex(pd.date_range("2020-07-01 00:00:00", "2021-07-01 23:00:00",freq='6h'), fill_value=np.nan)

In [325]:
testPM = pd.read_csv(os.path.join(folderpath, f"test\\{province[idx_province].lower()}_test.csv")).rename({"Unnamed: 0": "date_time"}, axis=1)
testPM['date_time'] = pd.to_datetime(testPM['date_time'])
testPM = testPM.set_index('date_time')

In [326]:
li = []
best_order = (1, 0, 0) # best_order = (1, 0, 1)
best_seasonal_order = (0, 1, 1, 12)#(1, 1, 0, 12) Try (1, 0, 1, 12)
exog_columns = ['temp', 'wind speed', 'wind dir'] + ['Thailand'] # Hotspot + wind + temp
target = 'PM2.5'

result, results_exog = sarimax_training(best_order, best_seasonal_order, train_data, exog_columns, target)

In [327]:
li = []
update_result, update_results_exog = result, results_exog
for i in tqdm(range(len(test_data)-1)):
  update_result = update_result.append(test_data['PM2.5'].iloc[i:i+1], exog=test_data[exog_columns].iloc[i:i+1])
  for idx, exog in enumerate(exog_columns):
    update_results_exog[idx] = update_results_exog[idx].append(test_data[exog].iloc[i:i+1])
  
  li.append(predict_target(update_result, update_results_exog, exog_columns, steps=12))

100%|██████████| 1463/1463 [25:11<00:00,  1.03s/it]


In [328]:
df = pd.concat(li, keys=test_data.index)

In [329]:
df

2020-07-01 00:00:00  2020-07-01 06:00:00    13.389646
                     2020-07-01 12:00:00    17.970650
                     2020-07-01 18:00:00    22.888691
                     2020-07-02 00:00:00    13.549758
                     2020-07-02 06:00:00    20.697297
                                              ...    
2021-07-01 12:00:00  2021-07-03 12:00:00    16.788266
                     2021-07-03 18:00:00    16.442560
                     2021-07-04 00:00:00    16.989823
                     2021-07-04 06:00:00    17.509482
                     2021-07-04 12:00:00    17.160463
Name: predicted_mean, Length: 17556, dtype: float64

In [334]:
df.to_csv(f'{province[idx_province].lower()}_predict_test_best.csv')

# Predict

In [330]:
gt = []
for data in tqdm(df.iteritems()):
  try:
    gt.append(testPM.loc[data[0][1]].values[0])
  except:
    gt.append(np.NaN)
gt = np.array(gt)

17556it [00:01, 13346.76it/s]


In [331]:
df.values

array([13.38964611, 17.97064995, 22.88869099, ..., 16.98982324,
       17.50948175, 17.16046304])

In [332]:
mask = np.isnan(gt)
predicted = df.values[~mask]
gt_target = gt[~mask]

In [333]:
mse = mean_squared_error(gt_target, predicted)
rmse = math.sqrt(mse)
print('Test on SARIMAX with RMSE: %f' % (rmse, ))

Test on SARIMAX with RMSE: 11.307005
